In [1]:
# LIBRARY IMPORT
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
#--------------------------------------------------------------------------------------#
START_YEAR                  = 1951 
END_YEAR                    = 2010 
#--------------------------------------------------------------------------------------#

In [3]:
# Unpack the dataset
ds = xr.open_dataset('Allstorms.ibtracs_all.v03r09.nc',decode_times=False)

In [4]:
INDEX_RECORD_SOURCE = 10 # JTWC_WP = 10

time_criteria = (ds.season <= END_YEAR) & (ds.season >= START_YEAR)
temp_matrix = np.ones([1,191])

storm           = np.ravel(np.multiply(temp_matrix,ds.storm[time_criteria].values[:,np.newaxis]))
season          = np.ravel(np.multiply(temp_matrix,ds.season[time_criteria].values[:,np.newaxis]))
genesis_basin   = np.ravel(np.multiply(temp_matrix,ds.genesis_basin[time_criteria].values[:,np.newaxis]))
basin           = np.ravel(np.multiply(temp_matrix,ds.basin[time_criteria].values[:,np.newaxis]))
source_time     = np.ravel(np.multiply(temp_matrix,ds.source_time[time_criteria].values[:,np.newaxis]))
source_lat      = np.ravel(ds.source_lat[time_criteria,:,INDEX_RECORD_SOURCE].values)
source_lon      = np.ravel(ds.source_lon[time_criteria,:,INDEX_RECORD_SOURCE].values)
source_wind     = np.ravel(ds.source_wind[time_criteria,:,INDEX_RECORD_SOURCE].values)
dist2land       = np.ravel(np.multiply(temp_matrix,ds.dist2land[time_criteria].values[:,np.newaxis]))
landfall        = np.ravel(np.multiply(temp_matrix,ds.landfall[time_criteria].values[:,np.newaxis]))

In [5]:
dataset = pd.DataFrame({'CycNo':storm,'YEAR':season,'GENESIS':genesis_basin,'BASIN':basin,'S_TIME':source_time,
                        'LAT':source_lat,
                        'LON':source_lon,
                        'VMAX':source_wind,
                        'DIST2LAND':dist2land,
                        'LANDFALL':landfall
                       })

In [6]:
dataset.head()

,BASIN,CycNo,DIST2LAND,GENESIS,LANDFALL,LAT,LON,S_TIME,VMAX,YEAR
0,4.0,5382.0,674.0,4.0,674.0,NaN,NaN,33512.00,NaN,1951.0
1,4.0,5382.0,815.0,4.0,815.0,NaN,NaN,33512.25,NaN,1951.0
2,4.0,5382.0,939.0,4.0,939.0,NaN,NaN,33512.50,NaN,1951.0
3,4.0,5382.0,1049.0,4.0,1049.0,NaN,NaN,33512.75,NaN,1951.0
4,4.0,5382.0,1149.0,4.0,1149.0,NaN,NaN,33513.00,NaN,1951.0


In [7]:
# Remove data points with null latitude
dataset = dataset[dataset['LAT'].notnull()]

# Remove data points with null vmax
dataset = dataset[dataset['VMAX'].notnull()]

# Remove data points from different basins
dataset = dataset[dataset['BASIN'] == 2]

In [8]:
print('Number of cyclones found: {}'.format(dataset['CycNo'].nunique()))

Number of cyclones found: 1707


In [9]:
dataset.to_csv('data_ibtracs_JTWC.csv', index=False)